In [ ]:
import textblob   
import pandas as pd
import csv
import time
import requests
from requests import get
from bs4 import BeautifulSoup

from time import sleep
from random import randint
import numpy as np     

import matplotlib.pyplot as plt
headers = {"Accept-Language": "en-US, en;q=0.5"}

Base_url = "https://uk.reuters.com"

#from reuters oil archive
pages = [str(i) for i in range(1,2000)]
url_list=[]

for page in pages:

    url= 'https://uk.reuters.com/news/archive/oilRpt?view=page&page='+page +'&pageSize=10'
    url_list.append(url)

print(url_list)

In [ ]:
from warnings import warn



In [ ]:
from IPython.core.display import clear_output
from time import time

start_time = time()
requests = 0

for _ in range(5):
    
    requests += 1
    sleep(randint(1,3))
    current_time = time()
    elapsed_time = current_time - start_time
    print('Request: {}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)

In [ ]:
# Create an empty list which will contain the selected news articles 
List_of_links = [] 
word1="article"

for newsurl in url_list:
    response = get(newsurl, headers = headers)
    
    sleep(randint(1,3))
    
    requests += 1
    elapsed_time = time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)
    
    # Throw a warning for non-200 status codes
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))
    
    # Parse the content of the request with BeautifulSoup
    #page_html = BeautifulSoup(response.text, 'html.parser')
    html_soup = BeautifulSoup(response.text, 'html.parser')
    sub_links = html_soup.find_all('a')
    for links in sub_links:
        sp = BeautifulSoup(str(links),'html.parser')  # first convert into a string
        tag = sp.a
      #if word1 in tag['title'] or word2 in tag['title'] or word3 in tag['title']:
        if word1 in tag['href']:
            category_links =  Base_url + tag["href"]
            List_of_links.append(category_links)
            
    
    

In [ ]:
unique_links = list(set(List_of_links))
for q in unique_links: print(q)


In [ ]:
df = pd.DataFrame({
                   'url_news'  : unique_links,
                  })

print(df.info())
df

In [ ]:
df.to_csv('links.csv')

In [ ]:
#read in links

dflinks=pd.read_csv('C:/Users/neil.watt/Documents/PythonScripts/Sentiment/Main/MarketSentiment_Analysis/links.csv')
dflinks['url_news']


In [ ]:


unique_links=dflinks['url_news']
# Create an empty list which will contain the selected news articles 
text_body=[]
date=[]


for selected_links in unique_links:
    response = get(selected_links, headers = headers)
    sleep(randint(1,3))
    requests += 1
    elapsed_time = time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)
      # Throw a warning for non-200 status codes
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))

        
    html_soup = BeautifulSoup(response.text, 'html.parser')
    
    #retrieve body of article
    results_text = html_soup.find(class_='StandardArticleBody_body')
    story_text = results_text.get_text()
    lower_story_text=story_text.lower()
    text_body.append(lower_story_text) 
    #retrieve date
    date_results_text = html_soup.find(class_='ArticleHeader_date')
    date_text = date_results_text.get_text()
    date.append(date_text) 
    
    
    
    


In [ ]:
df_articles.to_csv('df_articles.csv')